# Automated Mail Sender System in Python
The project aims to streamline the process of monitoring employee performance by automating the identification of underperformance and the subsequent sending of warning emails. This system utilizes a Google Sheet to store and manage employee performance data, evaluates the data against predefined performance targets, and automatically generates and sends personalized warning emails to employees who do not meet the set criteria.

In [ ]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import win32com.client as win32

In [ ]:
# Set up Google Sheets API credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("C:/Users/Lenovo/Desktop/hybrid-creek-431418-h8-e329e9e00ef5.json", scope)
client = gspread.authorize(creds)

# Open the Google Sheet
sheet = client.open_by_url("https://docs.google.com/spreadsheets/d/14LHEDphIyv2QyqE15ZboKum99tc6Izmfn1Thr8wa-mM/edit?gid=0#gid=0")
worksheet = sheet.get_worksheet(0)

# Convert Google Sheet data to DataFrame
data = worksheet.get_all_records()
df = pd.DataFrame(data)


In [ ]:
# Function to determine background color based on comparison
def get_background_color(metric, target, actual):
    try:
        target = float(target.replace('%', '').replace('₹', '').replace(',', ''))
    except ValueError:
        return '#ffffff'  # Default color if target cannot be converted

    try:
        actual = float(actual.replace('%', '').replace('₹', '').replace(',', ''))
    except ValueError:
        return '#ffffff'  # Default color if actual cannot be converted

    if metric == 'GMV (Gross Merchandise Value)':
        if actual > target:
            return '#d4edda'  # Light green
        elif actual < target - 100000:
            return '#FF4500'  # Light red
        else:
            return '#fff3cd'  # Yellow
    elif metric == 'Lead Conversion Rates (Non-Prepaid)':
        if actual > target:
            return '#d4edda'  # Light green
        elif actual < target - 2:
            return '#FF4500'  # Light red
        else:
            return '#fff3cd'  # Yellow
    elif metric == 'Average Ticket Size':
        if actual < target - 1000:
            return '#FF4500'  # Light red
        elif actual > target:
            return '#d4edda'  # Light green
        else:
            return '#fff3cd'  # Yellow
    elif metric == "warranty_sold":
        if actual < target -1:
            return "#FF4500"
        elif actual > target:
            return "#d4edda"
        else:
            return '#fff3cd'
    else:
        if actual < target - 5:
            return '#FF4500'  # Light red
        elif actual > target:
            return '#d4edda'  # Light green
        else:
            return '#fff3cd'  # Yellow


In [ ]:
# Function to generate email and html code that make a body of mail
def generate_email(row):
    metrics = [
        ('Order Conversion Rates (Prepaid)', '85%', f"{row['order_prepaid_actual']}%"),
        ('Order Conversion Rates (Non-Prepaid)', '50%', f"{row['order_Non_prepaid_actual']}%"),
        ('Lead Conversion Rates (Prepaid)', '70%', f"{row['Lead_prepaid_actual']}%"),
        ('Lead Conversion Rates (Non-Prepaid)', '14%', f"{row['lead_non_prepaid_actual']}%"),
        ('GMV (Gross Merchandise Value)', f"₹{row['GMV target']}", f"₹{row['GMV achievement']}"),
        ('Average Ticket Size', f"₹{row['target_ticket_size']}", f"₹{row['achievement_ticket_size']}"),
        ('Warranty Sold', f"{row['target_warranty_sold']}", f"{row['achievement_warrantly_sold']}")
    ]

    rows = ""
    for metric, target, actual in metrics:
        color = get_background_color(metric, target, actual)
        rows += f"""
        <tr>
            <td>{metric}</td>
            <td>{target}</td>
            <td style="background-color:{color}">{actual}</td>
        </tr>
        """

    subject = f"Urgent: Warning Mail | {row['Employee_name']}"
    body = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Urgent: Warning Mail | {row['Employee_name']}</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                color: #333;
                margin: 0;
                padding: 0;
            }}
            .container {{
                width: 80%;
                margin: auto;
                padding: 20px;
                border: 1px solid #e0e0e0;
                background-color: #f9f9f9;
                border-radius: 5px;
            }}
            .header {{
                background-color: #d9534f;
                color: #fff;
                padding: 10px;
                text-align: center;
                border-radius: 5px 5px 0 0;
            }}
            .content {{
                padding: 20px;
            }}
            .warning {{
                color: #d9534f;
                font-weight: bold;
            }}
            .table {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}
            .table th, .table td {{
                padding: 8px;
            }}
            .table th {{
                background-color: #d9534f;
                color: #fff;
                border-bottom: 2px solid #ddd;
            }}
            .table td {{
                text-align: center;
                border-bottom: 1px solid #ddd;  /* Adjust this as needed to keep table lines */
            }}
            .footer {{
                margin-top: 20px;
                text-align: center;
                font-size: 0.9em;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>Urgent: Warning Mail | {row['Employee_name']}</h1>
            </div>
            <div class="content">
                <p>Dear {row['Employee_name']},</p>
                <p>I hope this message finds you well.</p>
                <p>We are writing to address some concerns regarding your recent performance metrics. As you are aware, our team has set specific targets for order and lead conversions, as well as GMV (Gross Merchandise Value) and average ticket size, to ensure we meet our operational goals and deliver exceptional service to our customers. However, your current performance has not met these targets, and we want to bring this to your attention for immediate improvement.</p>
                <h2 class="warning">Current Performance vs. Targets:</h2>
                <table class="table">
                    <thead>
                        <tr>
                            <th>Metric</th>
                            <th>Target</th>
                            <th>Actual</th>
                        </tr>
                    </thead>
                    <tbody>
                        {rows}
                    </tbody>
                </table>
                <h2 class="warning">Immediate Action Required:</h2>
                <ul>
                    <li><strong>Review and Improvement Plan:</strong> Please review your recent performance data and identify areas where improvements can be made.</li>
                    <li><strong>Training and Support:</strong> If you require additional training or support, please let us know so we can arrange the necessary resources.</li>
                </ul>
                <p>Failure to demonstrate significant improvement in these areas may result in further disciplinary action, including the initiation of a Performance Improvement Plan (PIP).</p>
                <p>We value your contributions to the team and believe that with the right focus and effort, you can improve your performance. Please consider this as a formal warning and an opportunity to rectify the situation. We are confident that with a concerted effort, we can work together to achieve the desired results.</p>
                <p>Should you have any questions or need further clarification, feel free to reach out to us directly.</p>
                <p>Thank you for your immediate attention to this matter.</p>
            </div>
            <div class="footer">
              <p style="margin-bottom: 0;">Best regards,</p>
              <p style="margin-top: 0;">Central Team <strong>Go<span style="color: red;">Mechanic</span></strong></p>
            </div>
        </div>
    </body> 
    </html>
    """
    return subject, body


In [ ]:
# Function to send email through Outlook
def send_email(to_email, subject, body, cc_emails):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to_email
    mail.CC = ";".join(cc_emails)
    mail.Subject = subject
    mail.HTMLBody = body  # Set the email body to HTML
    mail.Send()


In [ ]:
# Generate and send emails
emails = df.apply(generate_email, axis=1)

for index, (subject, body) in emails.iteritems():
    to_email = df.loc[index, 'employee_email']
    cc_emails = [df.loc[index, 'cc_mail_1'], df.loc[index, 'cc_mail_2'], df.loc[index, 'cc_mail_3'], df.loc[index, 'cc_mail_4'], df.loc[index, 'cc_mail_5']]
    send_email(to_email, subject, body, cc_emails)
    print(f"Email sent to: {to_email}\nSubject: {subject}\n")


## Components and Workflow
### Google Sheet for Data Storage:

Data Entry: A Google Sheet is used to store employee performance data. Each row in the sheet represents an individual employee, and columns include various performance metrics, employee contact information, and target values.
Example Columns: Employee ID, Name, Email, Sales Target, Actual Sales, Customer Satisfaction Target, Actual Customer Satisfaction, etc.

### Data Retrieval:
Google Sheets API: The system uses the Google Sheets API to access and read data from the Google Sheet. This allows the script to programmatically retrieve the latest performance data.

### Data Processing and Analysis:
Discrepancy Identification: 
The system compares the actual performance metrics against the predefined targets. If discrepancies are found (i.e., if the actual performance is below the target), the employee is flagged for a warning email.
Example Logic: If Actual Sales < Sales Target or Actual Customer Satisfaction < Customer Satisfaction Target, the employee is marked as underperforming.

### Email Generation:

Personalization: The script generates personalized emails for each flagged employee. The content of the email includes the employee's name, the specific metrics they did not meet, and a message about the importance of meeting performance targets.
Email Template: A pre-defined email template is used, with placeholders for dynamic content such as employee name and specific performance details.

### Email Sending:
Email API: The system uses an email API (such as Gmail API or a service like SendGrid) to send the generated emails automatically.
Sending Mechanism: The script iterates over the list of flagged employees, fills in the email template with personalized details, and sends the email to the respective employee’s email address.